<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Sebastián Enríquez &bull; 2024-02
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [24]:
# Paquetes necesarios
from sklearn.datasets import fetch_openml
import pandas as pd  # Manejo de datos
import matplotlib.pyplot as plt  # Visualización
from sklearn.preprocessing import StandardScaler  # Estandarización de datos
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import train_test_split # División de datos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métrica de evaluación

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
import joblib

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [25]:
# Cargar los datos
data = pd.read_csv('https://raw.githubusercontent.com/BetoBetin56734/BaseAprendizajeAutomatico/refs/heads/main/charcters_stats.csv')

# Seleccionar columnas de interés
numeric_cols = ['Intelligence', 'Strength', 'Durability', 'Power', 'Combat']
categorical_features = ['Alignment']
target_col = 'Speed'

# Verificar valores de la variable objetivo
print("📊 Resumen estadístico de Speed:")
print(data[target_col].describe())

📊 Resumen estadístico de Speed:
count    611.000000
mean      27.306056
std       25.760173
min        1.000000
25%        1.000000
50%       23.000000
75%       42.000000
max      100.000000
Name: Speed, dtype: float64


In [26]:
data.describe()

,Intelligence,Strength,Speed,Durability,Power,Combat,Total
count,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000
mean,44.502455,29.083470,27.306056,41.839607,40.312602,43.206219,226.250409
std,32.924722,32.610939,25.760173,36.543849,34.539910,33.270329,167.324034
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,5.000000
25%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,5.000000
50%,50.000000,10.000000,23.000000,32.000000,37.000000,50.000000,255.000000
75%,75.000000,53.000000,42.000000,80.000000,67.000000,70.000000,351.500000
max,113.000000,100.000000,100.000000,120.000000,100.000000,101.000000,581.000000



### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [27]:
# Codificar la variable categórica
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

# Preparar X (características) y y (variable objetivo)
X = data[numeric_cols + ['Alignment_good', 'Alignment_neutral']]
y = data[target_col]

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo base
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"📉 Error cuadrático medio (MSE) del modelo base: {mse:.2f}")


📉 Error cuadrático medio (MSE) del modelo base: 304.47


In [28]:
print("🔧 Hiperparámetros del modelo Random Forest:")
print(model.get_params())

🔧 Hiperparámetros del modelo Random Forest:
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [29]:
# Definir los valores para optimización
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error')

# Ajustar modelo con búsqueda
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros y su desempeño
print("✔️ Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred_optimized = best_model.predict(X_test)
optimized_mse = mean_squared_error(y_test, y_pred_optimized)

print(f"📉 Error cuadrático medio (MSE) del modelo optimizado: {optimized_mse:.2f}")


✔️ Mejores hiperparámetros encontrados:
{'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 50}
📉 Error cuadrático medio (MSE) del modelo optimizado: 287.83


### <font color='264CC7'> Optimización por GridSearch </font>





<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [30]:
# Definir los valores para optimización
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10]
}

In [31]:
# Configurar GridSearchCV con validación cruzada (5 folds)
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,  # Número de folds
    scoring='neg_mean_squared_error',  # Optimizar MSE
    verbose=1,  # Mostrar progreso
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Ajustar el modelo
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, None],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error', verbose=1)

In [32]:
# Obtener los mejores hiperparámetros y su puntaje
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Convertir de negativo a positivo

print(f"✔️ Mejores hiperparámetros: {best_params}")
print(f"📉 Mejor MSE promedio en validación cruzada: {best_score:.2f}")

# Evaluar el modelo optimizado en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred_optimized = best_model.predict(X_test)
optimized_mse = mean_squared_error(y_test, y_pred_optimized)

print(f"📊 MSE en el conjunto de prueba con el modelo optimizado: {optimized_mse:.2f}")

✔️ Mejores hiperparámetros: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 50}
📉 Mejor MSE promedio en validación cruzada: 283.39
📊 MSE en el conjunto de prueba con el modelo optimizado: 287.83


### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [33]:
# Configurar los valores para optimización
param_dist = {
    'n_estimators': [50, 100, 150, 200, 300],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

In [34]:
# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=25,  # Número de iteraciones
    cv=5,  # Validación cruzada con 5 folds
    scoring='neg_mean_squared_error',  # Optimizar el MSE
    verbose=1,  # Mostrar progreso
    random_state=42,  # Reproducibilidad
    n_jobs=-1  # Usar todos los núcleos disponibles
)

In [35]:
# Ajustar modelo con RandomizedSearchCV
random_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y su puntaje
best_params = random_search.best_params_
best_score = -random_search.best_score_  # Convertir de negativo a positivo

print(f"✔️ Mejores hiperparámetros encontrados:")
print(best_params)
print(f"📉 Mejor MSE promedio en validación cruzada: {best_score:.2f}")

# Evaluar el modelo optimizado en el conjunto de prueba
best_model = random_search.best_estimator_
y_pred_optimized = best_model.predict(X_test)
optimized_mse = mean_squared_error(y_test, y_pred_optimized)

print(f"📊 MSE en el conjunto de prueba con el modelo optimizado: {optimized_mse:.2f}")

Fitting 5 folds for each of 25 candidates, totalling 125 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
15 fits failed out of a total of 125.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/

✔️ Mejores hiperparámetros encontrados:
{'n_estimators': 300, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}
📉 Mejor MSE promedio en validación cruzada: 276.39
📊 MSE en el conjunto de prueba con el modelo optimizado: 270.61


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [36]:
# Hiperparámetros óptimos de RandomizedSearchCV
best_params = {
    'n_estimators': 300,
    'min_samples_split': 8,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'max_depth': None
}

In [37]:
# Reentrenar el modelo con los parámetros óptimos
best_model = RandomForestRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred_optimized = best_model.predict(X_test)
optimized_mse = mean_squared_error(y_test, y_pred_optimized)

print(f"📊 MSE en el conjunto de prueba con el modelo reentrenado: {optimized_mse:.2f}")

# Guardar el modelo entrenado
joblib.dump(best_model, "best_random_forest_model.pkl")
print("✔️ Modelo guardado como 'best_random_forest_model.pkl'")


📊 MSE en el conjunto de prueba con el modelo reentrenado: 270.61
✔️ Modelo guardado como 'best_random_forest_model.pkl'


### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>